In [13]:
import gc
import warnings
warnings.filterwarnings('ignore')
import scipy as sp
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from tqdm.auto import tqdm
import itertools

### We load in the train data and preprocess

In [3]:
chunksize = 1000000 # How much of the data to load in

# Load in using X_cols and customer_ID
train_df_iter = pd.read_csv("train_data.csv", chunksize=chunksize) # for this notebook we will use all the data


train_df = pd.DataFrame()
for i_chunk, chunk in enumerate(train_df_iter): # i_chunk is the index of the chunk, and chunk is the actual data
    train_df = pd.concat([train_df, chunk]) # adding it to train_df which is an empty dataframe
    print(train_df.shape)

(1000000, 190)
(2000000, 190)
(3000000, 190)
(4000000, 190)
(5000000, 190)
(5531451, 190)


In [1]:
# the features we will be using in a list
features = train_df.drop(['customer_ID', 'S_2'], axis = 1).columns.to_list()

NameError: name 'train_df' is not defined

In [5]:
# This is the categorical features
cat_features = [
        "B_30",
        "B_38",
        "D_114",
        "D_116",
        "D_117",
        "D_120",
        "D_126",
        "D_63",
        "D_64",
        "D_66",
        "D_68",
    ]

In [6]:
# This is the number of features that (not including the categorical features), and there column names
num_features = [col for col in features if col not in cat_features]

In [7]:
print('Starting training feature engineer...')
train_num_agg = train_df.groupby("customer_ID")[num_features].agg(['mean', 'std', 'min', 'max', 'last'])
train_num_agg.columns = ['_'.join(x) for x in train_num_agg.columns]
train_num_agg.reset_index(inplace = True)
train_cat_agg = train_df.groupby("customer_ID")[cat_features].agg(['count', 'last', 'nunique'])
train_cat_agg.columns = ['_'.join(x) for x in train_cat_agg.columns]
train_cat_agg.reset_index(inplace = True)
train_labels = pd.read_csv('train_labels.csv')
train_df = train_num_agg.merge(train_cat_agg, how = 'inner', on = 'customer_ID').merge(train_labels, how = 'inner', on = 'customer_ID')
print("Finished training feature engineer")

Starting training feature engineer...
Finished training feature engineer


In [8]:
# Deleting the train numbers aggregated and the categorical ones aggregated
del train_num_agg, train_cat_agg

In [9]:
gc.collect() # NO CLUE WHAT THIS DOES

0

In [17]:
train_df.to_parquet("train_df_final.parquet")